In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid

# GCN

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, output_dim):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, output_dim)

    def forward(self, x, edge_index, mode=1):
        if mode == 1:
            x = F.dropout(x,p = 0.5 ,training=self.training)
            x = self.conv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x,p = 0.5 ,training=self.training)
            x = self.conv2(x, edge_index)
            x = F.dropout(x,p = 0.05 ,training=self.training)
            x = F.relu(x)
            return F.log_softmax(x, dim=1)
        elif mode == 2:
            x = F.dropout(x,p = 0.5 ,training=self.training)
            x = self.conv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x,p = 0.5 ,training=self.training)
            x = self.conv2(x, edge_index)
            x = F.dropout(x,p = 0.05 ,training=self.training)
            x = F.relu(x)
            return x

# GAT

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATConv, Linear
from torch_geometric.datasets import Planetoid


# Define the GAT model
class GAT(torch.nn.Module):
    def __init__(self, num_features, output_dim, embedding_dimension=32, num_heads=8):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_features, embedding_dimension, heads=num_heads, dropout=0.6)
        self.conv2 = GATConv(num_heads * embedding_dimension, output_dim, heads=num_heads, dropout=0.6)
        self.linear1 = Linear(embedding_dimension * 2 * num_heads, 32)
        # self.linear2 = Linear(32, 16)
        # self.linear3 = Linear(16, 8)

    def forward(self, x, edge_index, mode=1):
        if mode == 1:
            # Pass the input through the first GAT layer
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv1(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)
        elif mode == 2:
            # Pass the input through the first GAT layer
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv1(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv2(x, edge_index)

            return x


# GATv2

In [ ]:

import torch
import torch.nn.functional as F
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv, Linear
from torch_geometric.datasets import Planetoid


# Define the GAT model
class GATv2(torch.nn.Module):
    def __init__(self, num_features, output_dim, embedding_dimension=16, num_heads=8):
        super(GATv2, self).__init__()
        self.conv1 = GATv2Conv(num_features, embedding_dimension, heads=num_heads, dropout=0.6)
        self.conv2 = GATv2Conv(num_heads * embedding_dimension, embedding_dimension, heads=num_heads, dropout=0.6)
        self.linear1 = Linear(embedding_dimension * num_heads, output_dim)
        self.linear2 = Linear(embedding_dimension, output_dim)

    def forward(self, x, edge_index, mode=1):
        if mode == 1:
            # Pass the input through the first GAT layer
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv1(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)
        elif mode == 2:
            # Pass the input through the first GAT layer
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv1(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv2(x, edge_index)
            return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
device

device(type='cuda')

In [ ]:
# device = torch.device('cpu')
device

device(type='cuda')

# Create new masks

In [ ]:
import random


def make_random_mask(size, train_fraction, validation_fraction=0):
    new_order = random.sample(range(size), size)

    ls_train = np.zeros(size, dtype=bool)
    ls_validation = np.zeros(size, dtype=bool)
    ls_test = np.zeros(size, dtype=bool)

    for i, value in enumerate(new_order):
        if i < int(size * train_fraction):
            ls_train[value] = True
            ls_validation[value] = False
            ls_test[value] = False
        elif i < int(size * (train_fraction + validation_fraction)):
            ls_train[value] = False
            ls_validation[value] = True
            ls_test[value] = False
        else:
            ls_train[value] = False
            ls_validation[value] = False
            ls_test[value] = True

    ls_train = torch.from_numpy(ls_train)
    ls_validation = torch.from_numpy(ls_validation)
    ls_test = torch.from_numpy(ls_test)

    return ls_train, ls_validation, ls_test


In [ ]:
def train_GNN_mode_and_save(model, data, epochs, train_mask, test_mask, gnn_name="GNN_embeds.pt"):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.nll_loss(out[train_mask], data.y[train_mask])
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 50 == 0:
            model.eval()
            pred = model(data.x, data.edge_index).argmax(dim=1)
            correct = (pred[test_mask] == data.y[test_mask]).sum()
            acc = int(correct) / int(test_mask.sum())
            print(f'epoch : {epoch + 1} = >  Accuracy: {acc:.4f}')

    model.eval()
    pred = model(data.x, data.edge_index).argmax(dim=1)
    correct = (pred[test_mask] == data.y[test_mask]).sum()
    acc = int(correct) / int(test_mask.sum())
    print(f'Final accuracy: {acc:.4f}')

    # now save
    model.eval()
    pred_emb = model(data.x, data.edge_index, mode=2)
    torch.save(pred_emb, gnn_name)

    return acc, pred[test_mask], data.y[test_mask]


# Ecoder decodre

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
import math


In [ ]:
import torch.nn.functional as F


# Define autoencoder
class SymmetricAutoencoder(nn.Module):
    def __init__(self, input_size, embedding_size):
        super(SymmetricAutoencoder, self).__init__()

        self.encoder_layers = []
        self.decoder_layers = []

        # Encoder
        size = input_size
        while size > embedding_size:
            next_size = 2 ** int(math.log2(size) - 1)
            self.encoder_layers.append(nn.Linear(size, next_size))
            self.encoder_layers.append(nn.ReLU())
            size = next_size

        self.embedding = nn.Linear(size, embedding_size)

        # Decoder

        size = embedding_size

        while size < input_size:
            self.decoder_layers.append(nn.Linear(size, size * 2))
            self.encoder_layers.append(nn.ReLU())
            size = size * 2

        self.decoder_layers.append(nn.Linear(size, input_size))

        self.encoder = nn.Sequential(*self.encoder_layers)
        self.decoder = nn.Sequential(*self.decoder_layers)

    def forward(self, x):
        encoded = self.encoder(x)
        embedded = self.embedding(encoded)
        decoded = self.decoder(embedded)
        return decoded



In [ ]:
def make_enc_embeds(data_features, num_features, num_epochs=5000, embedding_size=32, file_name="encoder_32.pt"):
    # Initialize model and optimizer
    model = SymmetricAutoencoder(num_features, embedding_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.00003)
    criterion = nn.MSELoss()

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        reconstructed = model(data_features)
        loss = criterion(reconstructed, data_features)
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    # Now you can use the encoder part of the trained autoencoder to get embeddings
    encoded_features = model.encoder(data_features)
    torch.save(encoded_features, file_name)

In [ ]:
import random
def split_data(dataset, num_each_class_train, num_val=500, num_test=1000):
    data = dataset[0]
    y = torch.detach(data.y).numpy()
    keys = [x for x in range(len(y))]
    y_dictionary = dict(zip(keys, y))
    score_list = set(y)
    dic_list = [[] for i in range(len(score_list))]

    for item in y_dictionary:
        dic_list[y_dictionary[item]].append(item)

    l = []
    for sample_list in (dic_list):
        s_list = random.sample(sample_list, num_each_class_train)
        l.extend(s_list)

    train_mask = [False for i in range(len(data.x))]
    for num in l:
        train_mask[num] = True
    mylist = [x for x in range(len(data.x))]
    mylist = [elt for elt in mylist if elt not in l]
    l1 = random.sample(mylist, num_val)
    mylist = [elt for elt in mylist if elt not in l1]
    val_mask = [False for i in range(len(data.x))]
    for num in l1:
        val_mask[num] = True

    l2 = random.sample(mylist, num_test)
    test_mask = [False for i in range(len(data.x))]
    for num in l2:
        test_mask[num] = True

    return train_mask, val_mask, test_mask

# load ,concat  and finalize

In [ ]:
class DNN(torch.nn.Module):
    def __init__(self, input_size, num_classes, hidden_layer=64):
        super().__init__()
        self.linear1 = Linear(input_size, hidden_layer)
        self.linear2 = Linear(hidden_layer, hidden_layer)
        self.linear3 = Linear(hidden_layer, num_classes)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.dropout(x, p=0.2)
        x = F.relu(self.linear2(x))
        x = F.dropout(x, p=0.2)
        x = F.relu(self.linear3(x))
        return F.log_softmax(x, dim=1)

In [ ]:
def finalizer_DNN(train_mask, test_mask, hidden_layer_DNN_size=256, repeat_threshold=0.85,lr=0.001, epochs=3000,
                  file_name_gnn_emb="GNN_embeds.pt",
                  file_name_enc_emb="encoder_emb32_cora.pt", validation_mask = None , early_stop=0.9):
    gnn_emb = torch.load(file_name_gnn_emb).to(device)
    enc_emb = torch.load(file_name_enc_emb).to(device)

    concated_data = torch.cat((gnn_emb, enc_emb), dim=1)

    input_size = concated_data.shape[1]
    model = DNN(input_size, dataset.num_classes, hidden_layer=hidden_layer_DNN_size).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(concated_data)
        loss = F.nll_loss(out[train_mask], data.y[train_mask])
        loss.backward()
        optimizer.step()

        if epoch % 50 == 0:
            model.eval()

            pred = model(concated_data).argmax(dim=1)
            correct = (pred[train_mask] == data.y[train_mask]).sum()
            acc_train = int(correct) / int(train_mask.sum())

            correct = (pred[test_mask] == data.y[test_mask]).sum()
            acc = int(correct) / int(test_mask.sum())
            print(f'epoch {epoch} ,Accuracy: {acc:.4f} , train acc : {acc_train}')

            if validation_mask is not None:
              correct = (pred[validation_mask] == data.y[validation_mask]).sum()
              acc_validation = int(correct) / int(validation_mask.sum())
              if  acc_validation > early_stop:
                break


    # calculate train accuracy
    pred = model(concated_data).argmax(dim=1)
    correct = (pred[new_train_mask] == data.y[new_train_mask]).sum()
    acc_train = int(correct) / int(new_train_mask.sum())

    if (acc_train < repeat_threshold):
        return finalizer_DNN(train_mask, test_mask, repeat_threshold=repeat_threshold, epochs=epochs,
                             file_name_gnn_emb=file_name_gnn_emb,
                             file_name_enc_emb=file_name_enc_emb)

    model.eval()
    pred = model(concated_data).argmax(dim=1)
    correct = (pred[test_mask] == data.y[test_mask]).sum()
    acc = int(correct) / int(test_mask.sum())
    print(f'Accuracy: {acc:.4f}')
    return acc, pred[test_mask], data.y[test_mask]


# ALL scenarios

In [ ]:
def print_details(acc_list, f1_score_list):
    print(f'Accuracy avg = {statistics.mean(acc_list)}')
    print(f'Accuracy deviation = {statistics.stdev(acc_list)}')
    print(f'F1 score(macro) avg = {statistics.mean(f1_score_list)}')
    print(f'F1 score(macro) deviation = {statistics.stdev(f1_score_list)}')

In [ ]:
import statistics
from sklearn.metrics import f1_score
import copy

def run_model(GNN_model , Gnn_epochs ,enc_address ,reply_threshold , head_epochs, validation_mask =None , early_stop = 0.9 ,hidden_layer_DNN_size= 256):
  f1_score_list_base = []
  acc_list_base = []

  f1_score_list = []
  acc_list = []
  for i in range(10):

      model = copy.deepcopy(GNN_model)
      print("*****")
      print(f'epoch : {i + 1} / 10')
      # model = GATv2(dataset.num_features, dataset.num_classes, embedding_dimension=32).to(device)
      # GNN_model = GAT(dataset.num_features, dataset.num_classes, embedding_dimension=128).to(device)
      # model = GCN(128, dataset.num_classes).to(device)
      acc_base, preds_base, labels_base = train_GNN_mode_and_save(model, data, Gnn_epochs, train_mask=new_train_mask,
                                                                  test_mask=new_test_mask)
      preds_base = preds_base.cpu().numpy()
      labels_base = labels_base.cpu().numpy()
      f1_score_base = f1_score(labels_base, preds_base, average='macro')

      f1_score_list_base.append(f1_score_base)
      acc_list_base.append(acc_base)

      acc, preds, labels = finalizer_DNN(train_mask=new_train_mask, epochs=head_epochs, test_mask=new_test_mask,
                                        repeat_threshold=reply_threshold,
                                        file_name_enc_emb=enc_address,
                                        validation_mask = validation_mask,
                                        early_stop = early_stop,
                                        hidden_layer_DNN_size = hidden_layer_DNN_size)

      preds = preds.cpu().numpy()
      labels = labels.cpu().numpy()

      print("base model")
      print(acc_base)
      print(f1_score_base)

      print("OurModel")
      print(acc)
      print(f1_score(labels, preds, average='macro'))
      f1_score_list.append(f1_score(labels, preds, average='macro'))
      acc_list.append(acc)


  print("+++***Fianal Result***+++")
  print("base")
  print_details(acc_list_base, f1_score_list_base)
  print("new model")
  print_details(acc_list, f1_score_list)

In [ ]:
from torch_geometric.utils import add_self_loops

In [ ]:
from torch_geometric.utils import add_self_loops

# BlogCatalog

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
from torch_geometric.data import Data
from torch_geometric.datasets import AttributedGraphDataset
import torch_geometric.transforms as T
import torch.nn.functional as F

name_data = 'BlogCatalog'
dataset = AttributedGraphDataset(root='/tmp/' + name_data, name=name_data)

# dataset.transform = T.NormalizeFeatures()
print(f"Number of Classes in {name_data}:", dataset.num_classes)
print(f"Number of Node Features in {name_data}:", dataset.num_node_features)
data = dataset[0]
data.edge_index = add_self_loops(data.edge_index, num_nodes=data.num_nodes)[0]

Number of Classes in BlogCatalog: 6
Number of Node Features in BlogCatalog: 8189


In [ ]:
# Prepare data
features = data.x  # Feature matrix
num_features = features.shape[1]

new_train_mask, new_valdation_mask, new_test_mask = make_random_mask(data.x.shape[0], 0.5, 0.2)
new_train_mask, new_test_mask.shape
#
# new_train_mask = data.train_mask
# new_test_mask = data.test_mask

data = data.to(device)


In [ ]:
make_enc_embeds(data.x, num_features, num_epochs=1000, embedding_size=64, file_name="encoder_emb_BlogCatalog64.pt")

Epoch [1/1000], Loss: 0.008797779679298401
Epoch [2/1000], Loss: 0.008658826351165771
Epoch [3/1000], Loss: 0.00854803342372179
Epoch [4/1000], Loss: 0.008452270179986954
Epoch [5/1000], Loss: 0.008364767767488956
Epoch [6/1000], Loss: 0.008282391354441643
Epoch [7/1000], Loss: 0.00820410717278719
Epoch [8/1000], Loss: 0.008130121976137161
Epoch [9/1000], Loss: 0.008061368018388748
Epoch [10/1000], Loss: 0.007999199442565441
Epoch [11/1000], Loss: 0.007945405319333076
Epoch [12/1000], Loss: 0.007902255281805992
Epoch [13/1000], Loss: 0.007871883921325207
Epoch [14/1000], Loss: 0.00785469077527523
Epoch [15/1000], Loss: 0.007847809232771397
Epoch [16/1000], Loss: 0.007845879532396793
Epoch [17/1000], Loss: 0.007844439707696438
Epoch [18/1000], Loss: 0.007841890677809715
Epoch [19/1000], Loss: 0.007838337682187557
Epoch [20/1000], Loss: 0.007834013551473618
Epoch [21/1000], Loss: 0.007829245179891586
Epoch [22/1000], Loss: 0.007824593223631382
Epoch [23/1000], Loss: 0.00782056711614132
E

try vector size = 64

In [ ]:
GNN_model = GATv2(dataset.num_features, dataset.num_classes, embedding_dimension=8).to(device)
run_model(GNN_model , Gnn_epochs=700 ,enc_address="encoder_emb_BlogCatalog64.pt" ,reply_threshold=0.85 , head_epochs= 3000 , validation_mask=new_valdation_mask , early_stop=0.88 )

*****
epoch : 1 / 10
epoch : 50 = >  Accuracy: 0.1911
epoch : 100 = >  Accuracy: 0.3393
epoch : 150 = >  Accuracy: 0.4586
epoch : 200 = >  Accuracy: 0.5215
epoch : 250 = >  Accuracy: 0.5394
epoch : 300 = >  Accuracy: 0.5619
epoch : 350 = >  Accuracy: 0.5863
epoch : 400 = >  Accuracy: 0.6023
epoch : 450 = >  Accuracy: 0.6183
epoch : 500 = >  Accuracy: 0.6151
epoch : 550 = >  Accuracy: 0.6171
epoch : 600 = >  Accuracy: 0.6183
epoch : 650 = >  Accuracy: 0.6177
epoch : 700 = >  Accuracy: 0.6158
Final accuracy: 0.6158
epoch 0 ,Accuracy: 0.2322 , train acc : 0.21208622016936104
epoch 50 ,Accuracy: 0.6280 , train acc : 0.8210161662817552
epoch 100 ,Accuracy: 0.6639 , train acc : 0.8779830638953041
epoch 150 ,Accuracy: 0.6754 , train acc : 0.9053117782909931
epoch 200 ,Accuracy: 0.6870 , train acc : 0.9337952270977675
epoch 250 ,Accuracy: 0.6857 , train acc : 0.941108545034642
epoch 300 ,Accuracy: 0.6979 , train acc : 0.9515011547344111
epoch 350 ,Accuracy: 0.6915 , train acc : 0.9607390300230

In [ ]:
GNN_model = GAT(dataset.num_features, dataset.num_classes, embedding_dimension=8).to(device)
run_model(GNN_model , Gnn_epochs=700 ,enc_address="encoder_emb_BlogCatalog64.pt" ,reply_threshold=0.85 , head_epochs= 3000 , validation_mask=new_valdation_mask , early_stop=0.88 )

*****
epoch : 1 / 10
epoch : 50 = >  Accuracy: 0.2303
epoch : 100 = >  Accuracy: 0.3695
epoch : 150 = >  Accuracy: 0.4362
epoch : 200 = >  Accuracy: 0.4638
epoch : 250 = >  Accuracy: 0.4843
epoch : 300 = >  Accuracy: 0.5093
epoch : 350 = >  Accuracy: 0.5292
epoch : 400 = >  Accuracy: 0.5414
epoch : 450 = >  Accuracy: 0.5523
epoch : 500 = >  Accuracy: 0.5638
epoch : 550 = >  Accuracy: 0.5722
epoch : 600 = >  Accuracy: 0.5863
epoch : 650 = >  Accuracy: 0.5831
epoch : 700 = >  Accuracy: 0.5888
Final accuracy: 0.5888
epoch 0 ,Accuracy: 0.2373 , train acc : 0.24595842956120093
epoch 50 ,Accuracy: 0.5555 , train acc : 0.6354888375673595
epoch 100 ,Accuracy: 0.6722 , train acc : 0.8029253271747498
epoch 150 ,Accuracy: 0.7017 , train acc : 0.8406466512702079
epoch 200 ,Accuracy: 0.7171 , train acc : 0.8683602771362586
epoch 250 ,Accuracy: 0.7229 , train acc : 0.8899153194765204
epoch 300 ,Accuracy: 0.7191 , train acc : 0.8987682832948422
epoch 350 ,Accuracy: 0.7126 , train acc : 0.914549653579

In [ ]:
GNN_model = GCN(64, dataset.num_classes).to(device)
run_model(GNN_model , Gnn_epochs=700 ,enc_address="encoder_emb_BlogCatalog64.pt" ,reply_threshold=0.85 , head_epochs= 3000 , validation_mask=new_valdation_mask , early_stop=0.88 )

*****
epoch : 1 / 10
epoch : 50 = >  Accuracy: 0.3990
epoch : 100 = >  Accuracy: 0.4785
epoch : 150 = >  Accuracy: 0.5382
epoch : 200 = >  Accuracy: 0.5696
epoch : 250 = >  Accuracy: 0.5920
epoch : 300 = >  Accuracy: 0.6068
epoch : 350 = >  Accuracy: 0.6126
epoch : 400 = >  Accuracy: 0.6177
epoch : 450 = >  Accuracy: 0.6196
epoch : 500 = >  Accuracy: 0.6241
epoch : 550 = >  Accuracy: 0.6260
epoch : 600 = >  Accuracy: 0.6286
epoch : 650 = >  Accuracy: 0.6318
epoch : 700 = >  Accuracy: 0.6344
Final accuracy: 0.6344
epoch 0 ,Accuracy: 0.3675 , train acc : 0.3464203233256351
epoch 50 ,Accuracy: 0.6645 , train acc : 0.6689761354888376
epoch 100 ,Accuracy: 0.6998 , train acc : 0.6982294072363356
epoch 150 ,Accuracy: 0.6985 , train acc : 0.7109314857582756
epoch 200 ,Accuracy: 0.7030 , train acc : 0.7167051578137028
epoch 250 ,Accuracy: 0.7062 , train acc : 0.7217090069284064
epoch 300 ,Accuracy: 0.6972 , train acc : 0.7263279445727483
epoch 350 ,Accuracy: 0.7075 , train acc : 0.7220939183987